In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['video_create_log.txt', 'user_activity_log.txt', 'app_launch_log.txt', 'user_register_log.txt']


In [2]:
# Load dataset
register_df = pd.read_table('./input/user_register_log.txt',header=None, sep='\t',
                            names=['user_id','register_day','register_type','device_type'])
launch_df = pd.read_table('./input/app_launch_log.txt', names=['user_id','launch_day'],header=None, sep='\t')
video_df = pd.read_table('./input/video_create_log.txt', names=['user_id','create_day'],header=None, sep='\t')
activity_df = pd.read_table('./input/user_activity_log.txt', header=None, sep='\t',
                            names=['user_id','activity_day','page','video_id','author_id','action_type'])

In [3]:
# train dataset
## train x
tr_time = 24
tr_register_df = register_df[register_df.register_day<=tr_time]
tr_x_df = tr_register_df
## train y
tr_launch_uid = launch_df[launch_df.launch_day>tr_time].user_id.unique()
tr_video_uid = video_df[video_df.create_day>tr_time].user_id.unique()
tr_activity_uid = activity_df[activity_df.activity_day>tr_time].user_id.unique()
tr_y = tr_register_df.user_id.map(lambda x: x in tr_launch_uid or x in tr_activity_uid or x in tr_video_uid ).map({False: 0, True:1})

In [4]:
# register
# register type --> (0,1,2,3)
tr_x_df[tr_x_df.register_type>=3]['register_type'] = 3
# register: device type --> (0,1,2,3)
device_type_count = tr_x_df['device_type'].value_counts()
def device_map(x):
    if(device_type_count[x]>1500):
        return 0
    elif(device_type_count[x]>1000):
        return 1
    elif(device_type_count[x]>500):
        return 2
    else:
        return 3

tr_device_type = tr_x_df['device_type'].map(device_map)
tr_x_df['device_pop_type'] = tr_device_type

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# launch
tr_launch_df = launch_df[launch_df.launch_day<=tr_time]
# launch_bool
tr_launch_uid = tr_launch_df.user_id.unique()
tr_x_df['launch_bool'] = tr_register_df.user_id.map(lambda x: x in tr_launch_uid).map({False: 0, True:1})
# launch num (ununique)
tr_launch_df_rmdup = tr_launch_df.drop_duplicates()
tr_launch_num_df = tr_launch_df.groupby('user_id').count().reset_index().rename(columns={'launch_day': 'launch_num'})
# launch count(unique)
tr_launch_count_df = tr_launch_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'launch_day': 'launch_count'})
# launch ratio
tr_launch_ratio_df = pd.merge(tr_register_df[['user_id', 'register_day']],tr_launch_count_df, how='left', on='user_id' )
tr_x_df['launch_ratio'] = tr_launch_ratio_df.launch_count/(tr_time-tr_launch_ratio_df.register_day+1)
# launch first day interval
tr_launch_firstday_df = tr_launch_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'launch_day': 'launch_firstday_interval'})
tr_launch_firstday_interval = tr_launch_firstday_df['launch_firstday_interval']-tr_register_df[tr_x_df['launch_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
tr_launch_firstday_interval_df = pd.concat([tr_launch_firstday_df.user_id, tr_launch_firstday_interval], axis=1)
tr_launch_firstday_interval_df.columns=['user_id', 'launch_firstday_interval']
# launch last day interval
tr_launch_lastday_df = tr_launch_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'launch_day': 'launch_lastday_interval'})
tr_launch_lastday_interval = tr_time-tr_launch_lastday_df['launch_lastday_interval']
tr_launch_lastday_interval_df = pd.concat([tr_launch_lastday_df.user_id, tr_launch_lastday_interval], axis=1)
# launch interval day(max)
tr_launch_df_sort = tr_launch_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('launch_day', ascending=True)).reset_index(drop=True)
tr_launch_interval_df = pd.concat([tr_launch_df_sort['user_id'], tr_launch_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'launch_day': 'launch_interval_day'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [6]:
def fillna_tr_launch_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.launch_interval_day)):
            #print(tr_launch_firstday_df[tr_launch_firstday_df.user_id==row.user_id])
            #print(tr_launch_lastday_df[tr_launch_lastday_df.user_id==row.user_id])
            new_df.iloc[idx, 1] = max(int(tr_launch_firstday_df[tr_launch_firstday_df.user_id==row.user_id].launch_firstday_interval),
                                      int(tr_launch_lastday_df[tr_launch_lastday_df.user_id==row.user_id].launch_lastday_interval))
    return new_df
tr_launch_interval_df = fillna_tr_launch_interval(tr_launch_interval_df)

In [7]:
# video 
tr_video_df = video_df[video_df.create_day<=tr_time]
# video bool
tr_video_uid = tr_video_df.user_id.unique()
tr_x_df['create_bool']= tr_register_df.user_id.map(lambda x: x in tr_video_uid).map({False: 0, True:1})
# video num (ununique)
tr_video_df_rmdup = tr_video_df.drop_duplicates()
tr_video_num_df = tr_video_df.groupby('user_id').count().reset_index().rename(columns={'create_day': 'create_num'})
# video count (unique)
tr_video_count_df = tr_video_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'create_day': 'create_count'})
# video ratio 
tr_video_ratio_df = pd.merge(tr_register_df[['user_id', 'register_day']],tr_video_count_df, how='left', on='user_id' )
tr_x_df['create_ratio'] = tr_video_ratio_df.create_count/(tr_time-tr_video_ratio_df.register_day+1)
# video first day interval
tr_video_firstday_df = tr_video_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'create_day': 'create_firstday_interval'})
tr_video_firstday_interval = tr_video_firstday_df['create_firstday_interval']-tr_register_df[tr_x_df['create_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
tr_video_firstday_interval_df = pd.concat([tr_video_firstday_df.user_id, tr_video_firstday_interval], axis=1)
tr_video_firstday_interval_df.columns = ['user_id', 'create_firstday_interval']
# video last day interval
tr_video_lastday_df = tr_video_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'create_day': 'create_lastday_interval'})
tr_video_lastday_interval = tr_time-tr_video_lastday_df['create_lastday_interval']
tr_video_lastday_interval_df = pd.concat([tr_video_lastday_df.user_id, tr_video_lastday_interval], axis=1)
# video interval day (max)
tr_video_df_sort = tr_video_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('create_day', ascending=True)).reset_index(drop=True)
tr_video_interval_df = pd.concat([tr_video_df_sort['user_id'], tr_video_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'create_day': 'create_interval_day'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [8]:
def fillna_tr_video_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.create_interval_day)):
            new_df.iloc[idx, 1] = max(int(tr_video_firstday_df[tr_video_firstday_df.user_id==row.user_id].create_firstday_interval),
                                      int(tr_video_lastday_df[tr_video_lastday_df.user_id==row.user_id].create_lastday_interval))
    return new_df
tr_video_interval_df = fillna_tr_video_interval(tr_video_interval_df)

In [9]:
# acitivity 
tr_activity_df = activity_df[activity_df.activity_day<=tr_time]
# activity bool
tr_activity_uid = tr_activity_df.user_id.unique()
tr_x_df['activity_bool']= tr_register_df.user_id.apply(lambda x: x in tr_activity_uid).map({False: 0, True:1})
# acitivity num(ununique)
tr_activity_df_rmdup = tr_activity_df[['user_id','activity_day']].drop_duplicates()
tr_activity_num_df = tr_activity_df[['user_id','activity_day']].groupby('user_id').count().reset_index().rename(columns={'activity_day': 'activity_num'})
# acitivity count(unique)
tr_activity_count_df = tr_activity_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'activity_day': 'activity_count'})
# activity ratio 
tr_activity_ratio_df = pd.merge(tr_register_df[['user_id', 'register_day']],tr_activity_count_df, how='left', on='user_id' )
tr_x_df['activity_ratio'] = tr_activity_ratio_df.activity_count/(tr_time-tr_activity_ratio_df.register_day+1)
# activity first day interval
tr_activity_firstday_df = tr_activity_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'activity_day': 'activity_firstday_interval'})
tr_activity_firstday_interval = tr_activity_firstday_df['activity_firstday_interval']-tr_register_df[tr_x_df['activity_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
tr_activity_firstday_interval_df = pd.concat([tr_activity_firstday_df.user_id, tr_activity_firstday_interval], axis=1)
tr_activity_firstday_interval_df.columns = ['user_id', 'activity_firstday_interval']
# activity last day interval
tr_activity_lastday_df = tr_activity_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'activity_day': 'activity_lastday_interval'})
tr_activity_lastday_interval = tr_time-tr_activity_lastday_df['activity_lastday_interval']
tr_activity_lastday_interval_df = pd.concat([tr_activity_lastday_df.user_id, tr_activity_lastday_interval], axis=1)
# activity interval day (max)
tr_activity_df_sort = tr_activity_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('activity_day', ascending=True)).reset_index(drop=True)
tr_activity_interval_df = pd.concat([tr_activity_df_sort['user_id'], tr_activity_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'activity_day': 'activity_interval_day'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [10]:
def fillna_tr_activity_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.activity_interval_day)):
            new_df.iloc[idx, 1] = max(int(tr_activity_firstday_df[tr_activity_firstday_df.user_id==row.user_id].activity_firstday_interval),
                                      int(tr_activity_lastday_df[tr_activity_lastday_df.user_id==row.user_id].activity_lastday_interval))
    return new_df
tr_activity_interval_df = fillna_tr_activity_interval(tr_activity_interval_df)

In [11]:
tr_page_df = tr_activity_df[['user_id','page']]
tr_page_df['bool'] = 1
tr_new_page_df = tr_page_df.groupby(['user_id', 'page']).sum().reset_index()
tr_page_dummies_df = pd.DataFrame(index=tr_new_page_df.user_id.unique())
tr_x_page=(tr_page_df.groupby(['user_id', 'page']).sum()/tr_page_df.groupby(['user_id']).sum()).drop(['page'], axis=1).reset_index()
for idx,row in tr_x_page.iterrows():
    tr_page_dummies_df.loc[int(row.user_id), 'page_'+str(int(row.page))]=float(row['bool'])
tr_page_dummies_df = tr_page_dummies_df[np.sort(np.asarray(tr_page_dummies_df.columns))]
tr_page_dummies_df.reset_index(inplace=True)
tr_page_dummies_df.rename(columns={'index':'user_id'},inplace=True)
tr_page_dummies_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,page_0,page_1,page_2,page_3,page_4
0,16,0.849515,NaN,0.038835,0.111650,NaN
1,30,0.562500,NaN,NaN,0.437500,NaN
2,98,NaN,NaN,1.000000,NaN,NaN
3,105,0.969512,0.001355,0.029133,NaN,NaN
4,218,0.204461,0.211896,0.001239,0.578686,0.003717


In [12]:
tr_action_type_df = tr_activity_df[['user_id','action_type']]
tr_action_type_df['bool'] = 1
tr_new_action_type_df = tr_action_type_df.groupby(['user_id', 'action_type']).sum().reset_index()
tr_action_type_dummies_df = pd.DataFrame(index=tr_new_action_type_df.user_id.unique())
tr_x_action_type=(tr_action_type_df.groupby(['user_id', 'action_type']).sum()/tr_action_type_df.groupby(['user_id']).sum()).drop(['action_type'], axis=1).reset_index()
for idx,row in tr_x_action_type.iterrows():
    tr_action_type_dummies_df.loc[int(row.user_id), 'action_type_'+str(int(row.action_type))]=float(row['bool'])
tr_action_type_dummies_df = tr_action_type_dummies_df[np.sort(np.asarray(tr_action_type_dummies_df.columns))]
tr_action_type_dummies_df.reset_index(inplace=True)
tr_action_type_dummies_df.rename(columns={'index':'user_id'},inplace=True)
tr_action_type_dummies_df.head()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_type_4,action_type_5
0,16,0.980583,0.009709,0.009709,NaN,NaN,NaN
1,30,0.964286,0.008929,0.026786,NaN,NaN,NaN
2,98,1.000000,NaN,NaN,NaN,NaN,NaN
3,105,0.927507,0.001355,0.000678,0.070461,NaN,NaN
4,218,0.976456,0.003717,0.018587,0.001239,NaN,NaN


In [13]:
tr_author_id_df = tr_activity_df[['user_id', 'author_id']]
tr_author_id_df_rmdup = tr_author_id_df[['user_id','author_id']].drop_duplicates()
tr_author_id_count_df = tr_author_id_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'author_id': 'author_id_count'})

tr_video_id_df = tr_activity_df[['user_id', 'video_id']]
tr_video_id_df_rmdup = tr_video_id_df[['user_id','video_id']].drop_duplicates()
tr_video_id_count_df = tr_video_id_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'video_id': 'video_id_count'})

tr_video_author_id_df = pd.merge(tr_video_id_count_df,tr_author_id_count_df, how='left',on='user_id')
tr_video_author_id_df['video_author_ratio'] = tr_video_author_id_df['video_id_count']/tr_video_author_id_df['author_id_count']
tr_video_author_id_df.head()



,user_id,video_id_count,author_id_count,video_author_ratio
0,16,187,162,1.154321
1,30,97,56,1.732143
2,98,1,1,1.000000
3,105,1212,1015,1.194089
4,218,633,149,4.248322


In [14]:
# merge register, video, activity

tr_x_df = pd.merge(tr_x_df, tr_launch_num_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_launch_count_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_launch_firstday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_launch_lastday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_launch_interval_df, how='left', on='user_id')

tr_x_df = pd.merge(tr_x_df, tr_video_num_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_video_count_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_video_firstday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_video_lastday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_video_interval_df, how='left', on='user_id')

tr_x_df = pd.merge(tr_x_df, tr_activity_num_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_activity_count_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_activity_firstday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_activity_lastday_interval_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_activity_interval_df, how='left', on='user_id')
#print(tr_x_df.shape)
tr_x_df = pd.merge(tr_x_df, tr_page_dummies_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_action_type_dummies_df, how='left', on='user_id')
tr_x_df = pd.merge(tr_x_df, tr_video_author_id_df, how='left', on='user_id')

(40518, 26)


In [17]:
tr_x_df[['launch_firstday_interval', 'launch_lastday_interval', 'launch_interval_day',]].fillna(tr_time, inplace=True)
tr_x_df[['create_firstday_interval', 'create_lastday_interval', 'create_interval_day',]].fillna(tr_time, inplace=True)
tr_x_df[['activity_firstday_interval', 'activity_lastday_interval', 'activity_interval_day']].fillna(tr_time, inplace=True)
tr_x_df.fillna(0, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [19]:
# fill null value with 0 , get_dummie and drop columns
#launch interval day (null value)
tr_x_df = pd.concat([tr_x_df, pd.get_dummies(tr_x_df['register_type'], prefix='register_type')], axis=1)
tr_x_df = pd.concat([tr_x_df, pd.get_dummies(tr_x_df['device_pop_type'], prefix='device_pop_type')], axis=1)
#tr_x_df = pd.concat([tr_x_df, pd.get_dummies(tr_x_df['device_pop_type'], prefix='device_pop_type')], axis=1)
tr_x_df = tr_x_df.drop(['user_id', 'device_type', 'device_pop_type','register_type'], axis=1)
tr_x_df.head()

,register_day,launch_bool,launch_ratio,create_bool,create_ratio,activity_bool,activity_ratio,launch_num,launch_count,launch_firstday_interval,...,register_type_6,register_type_7,register_type_8,register_type_9,register_type_10,register_type_11,device_pop_type_0,device_pop_type_1,device_pop_type_2,device_pop_type_3
0,1,1,0.125000,0,0.0,1,0.125000,3,3,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0.083333,0,0.0,1,0.083333,2,2,0,...,0,0,0,0,0,0,0,0,0,1
2,1,1,0.250000,0,0.0,1,0.250000,6,6,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1,0.541667,0,0.0,1,0.333333,13,13,0,...,0,0,0,0,0,0,0,0,0,1
4,1,1,0.416667,0,0.0,1,0.250000,10,10,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# test dataset
## test x
te_time = 30
te_register_df = register_df[register_df.register_day<=te_time]
te_x_df = te_register_df


# register
# register type --> (0,1,2,3)
te_x_df[te_x_df.register_type>=3]['register_type'] = 3
# register: device type --> (0,1,2,3)
device_type_count = te_x_df['device_type'].value_counts()
def device_map(x):
    if(device_type_count[x]>1500):
        return 0
    elif(device_type_count[x]>1000):
        return 1
    elif(device_type_count[x]>500):
        return 2
    else:
        return 3

te_device_type = te_x_df['device_type'].map(device_map)
te_x_df['device_pop_type'] = te_device_type

# launch
te_launch_df = launch_df[launch_df.launch_day<=te_time]
# launch_bool
te_launch_uid = te_launch_df.user_id.unique()
te_x_df['launch_bool'] = te_register_df.user_id.map(lambda x: x in te_launch_uid).map({False: 0, True:1})
# launch num (ununique)
te_launch_df_rmdup = te_launch_df.drop_duplicates()
te_launch_num_df = te_launch_df.groupby('user_id').count().reset_index().rename(columns={'launch_day': 'launch_num'})
# launch count(unique)
te_launch_count_df = te_launch_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'launch_day': 'launch_count'})
# launch ratio
te_launch_ratio_df = pd.merge(te_register_df[['user_id', 'register_day']],te_launch_count_df, how='left', on='user_id' )
te_x_df['launch_ratio'] = te_launch_ratio_df.launch_count/(te_time-te_launch_ratio_df.register_day+1)
# launch first day interval
te_launch_firstday_df = te_launch_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'launch_day': 'launch_firstday_interval'})
te_launch_firstday_interval = te_launch_firstday_df['launch_firstday_interval']-te_register_df[te_x_df['launch_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
te_launch_firstday_interval_df = pd.concat([te_launch_firstday_df.user_id, te_launch_firstday_interval], axis=1)
te_launch_firstday_interval_df.columns=['user_id', 'launch_firstday_interval']
# launch last day interval
te_launch_lastday_df = te_launch_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'launch_day': 'launch_lastday_interval'})
te_launch_lastday_interval = te_time-te_launch_lastday_df['launch_lastday_interval']
te_launch_lastday_interval_df = pd.concat([te_launch_lastday_df.user_id, te_launch_lastday_interval], axis=1)
# launch interval day(max)
te_launch_df_sort = te_launch_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('launch_day', ascending=True)).reset_index(drop=True)
te_launch_interval_df = pd.concat([te_launch_df_sort['user_id'], te_launch_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'launch_day': 'launch_interval_day'})

def fillna_te_launch_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.launch_interval_day)):
            #print(te_launch_firstday_df[te_launch_firstday_df.user_id==row.user_id])
            #print(te_launch_lastday_df[te_launch_lastday_df.user_id==row.user_id])
            new_df.iloc[idx, 1] = max(int(te_launch_firstday_df[te_launch_firstday_df.user_id==row.user_id].launch_firstday_interval),
                                      int(te_launch_lastday_df[te_launch_lastday_df.user_id==row.user_id].launch_lastday_interval))
    return new_df
te_launch_interval_df = fillna_te_launch_interval(te_launch_interval_df)

# video 
te_video_df = video_df[video_df.create_day<=te_time]
# video bool
te_video_uid = te_video_df.user_id.unique()
te_x_df['create_bool']= te_register_df.user_id.map(lambda x: x in te_video_uid).map({False: 0, True:1})
# video num (ununique)
te_video_df_rmdup = te_video_df.drop_duplicates()
te_video_num_df = te_video_df.groupby('user_id').count().reset_index().rename(columns={'create_day': 'create_num'})
# video count (unique)
te_video_count_df = te_video_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'create_day': 'create_count'})
# video ratio 
te_video_ratio_df = pd.merge(te_register_df[['user_id', 'register_day']],te_video_count_df, how='left', on='user_id' )
te_x_df['create_ratio'] = te_video_ratio_df.create_count/(te_time-te_video_ratio_df.register_day+1)
# video first day interval
te_video_firstday_df = te_video_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'create_day': 'create_firstday_interval'})
te_video_firstday_interval = te_video_firstday_df['create_firstday_interval']-te_register_df[te_x_df['create_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
te_video_firstday_interval_df = pd.concat([te_video_firstday_df.user_id, te_video_firstday_interval], axis=1)
te_video_firstday_interval_df.columns = ['user_id', 'create_firstday_interval']
# video last day interval
te_video_lastday_df = te_video_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'create_day': 'create_lastday_interval'})
te_video_lastday_interval = te_time-te_video_lastday_df['create_lastday_interval']
te_video_lastday_interval_df = pd.concat([te_video_lastday_df.user_id, te_video_lastday_interval], axis=1)
# video interval day (max)
te_video_df_sort = te_video_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('create_day', ascending=True)).reset_index(drop=True)
te_video_interval_df = pd.concat([te_video_df_sort['user_id'], te_video_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'create_day': 'create_interval_day'})

def fillna_te_video_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.create_interval_day)):
            new_df.iloc[idx, 1] = max(int(te_video_firstday_df[te_video_firstday_df.user_id==row.user_id].create_firstday_interval),
                                      int(te_video_lastday_df[te_video_lastday_df.user_id==row.user_id].create_lastday_interval))
    return new_df
te_video_interval_df = fillna_te_video_interval(te_video_interval_df)

# acitivity 
te_activity_df = activity_df[activity_df.activity_day<=te_time]
# activity bool
te_activity_uid = te_activity_df.user_id.unique()
te_x_df['activity_bool']= te_register_df.user_id.apply(lambda x: x in te_activity_uid).map({False: 0, True:1})
# acitivity num(ununique)
te_activity_df_rmdup = te_activity_df[['user_id','activity_day']].drop_duplicates()
te_activity_num_df = te_activity_df[['user_id','activity_day']].groupby('user_id').count().reset_index().rename(columns={'activity_day': 'activity_num'})
# acitivity count(unique)
te_activity_count_df = te_activity_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'activity_day': 'activity_count'})
# activity ratio 
te_activity_ratio_df = pd.merge(te_register_df[['user_id', 'register_day']],te_activity_count_df, how='left', on='user_id' )
te_x_df['activity_ratio'] = te_activity_ratio_df.activity_count/(te_time-te_activity_ratio_df.register_day+1)
# activity first day interval
te_activity_firstday_df = te_activity_df_rmdup.groupby('user_id').min().reset_index().rename(columns={'activity_day': 'activity_firstday_interval'})
te_activity_firstday_interval = te_activity_firstday_df['activity_firstday_interval']-te_register_df[te_x_df['activity_bool']==1].sort_values(by='user_id').register_day.reset_index(drop=True)
te_activity_firstday_interval_df = pd.concat([te_activity_firstday_df.user_id, te_activity_firstday_interval], axis=1)
te_activity_firstday_interval_df.columns = ['user_id', 'activity_firstday_interval']
# activity last day interval
te_activity_lastday_df = te_activity_df_rmdup.groupby('user_id').max().reset_index().rename(columns={'activity_day': 'activity_lastday_interval'})
te_activity_lastday_interval = te_time-te_activity_lastday_df['activity_lastday_interval']
te_activity_lastday_interval_df = pd.concat([te_activity_lastday_df.user_id, te_activity_lastday_interval], axis=1)
# activity interval day (max)
te_activity_df_sort = te_activity_df_rmdup.groupby('user_id').apply(lambda x: x.sort_values('activity_day', ascending=True)).reset_index(drop=True)
te_activity_interval_df = pd.concat([te_activity_df_sort['user_id'], te_activity_df_sort.groupby('user_id').diff()], axis=1).groupby('user_id').max().reset_index().rename(columns={'activity_day': 'activity_interval_day'})

def fillna_te_activity_interval(df):
    new_df = df
    for idx, row in df.iterrows():
        if(pd.isna(row.activity_interval_day)):
            new_df.iloc[idx, 1] = max(int(te_activity_firstday_df[te_activity_firstday_df.user_id==row.user_id].activity_firstday_interval),
                                      int(te_activity_lastday_df[te_activity_lastday_df.user_id==row.user_id].activity_lastday_interval))
    return new_df
te_activity_interval_df = fillna_te_activity_interval(te_activity_interval_df)

te_page_df = te_activity_df[['user_id','page']]
te_page_df['bool'] = 1
te_new_page_df = te_page_df.groupby(['user_id', 'page']).sum().reset_index()
te_page_dummies_df = pd.DataFrame(index=te_new_page_df.user_id.unique())
te_x_page=(te_page_df.groupby(['user_id', 'page']).sum()/te_page_df.groupby(['user_id']).sum()).drop(['page'], axis=1).reset_index()
for idx,row in te_x_page.iterrows():
    te_page_dummies_df.loc[int(row.user_id), 'page_'+str(int(row.page))]=float(row['bool'])
te_page_dummies_df = te_page_dummies_df[np.sort(np.asarray(te_page_dummies_df.columns))]
te_page_dummies_df.reset_index(inplace=True)
te_page_dummies_df.rename(columns={'index':'user_id'},inplace=True)
#te_page_dummies_df.head()

te_action_type_df = te_activity_df[['user_id','action_type']]
te_action_type_df['bool'] = 1
te_new_action_type_df = te_action_type_df.groupby(['user_id', 'action_type']).sum().reset_index()
te_action_type_dummies_df = pd.DataFrame(index=te_new_action_type_df.user_id.unique())
te_x_action_type=(te_action_type_df.groupby(['user_id', 'action_type']).sum()/te_action_type_df.groupby(['user_id']).sum()).drop(['action_type'], axis=1).reset_index()
for idx,row in te_x_action_type.iterrows():
    te_action_type_dummies_df.loc[int(row.user_id), 'action_type_'+str(int(row.action_type))]=float(row['bool'])
te_action_type_dummies_df = te_action_type_dummies_df[np.sort(np.asarray(te_action_type_dummies_df.columns))]
te_action_type_dummies_df.reset_index(inplace=True)
te_action_type_dummies_df.rename(columns={'index':'user_id'},inplace=True)
#te_action_type_dummies_df.head()

te_author_id_df = te_activity_df[['user_id', 'author_id']]
te_author_id_df_rmdup = te_author_id_df[['user_id','author_id']].drop_duplicates()
te_author_id_count_df = te_author_id_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'author_id': 'author_id_count'})

te_video_id_df = te_activity_df[['user_id', 'video_id']]
te_video_id_df_rmdup = te_video_id_df[['user_id','video_id']].drop_duplicates()
te_video_id_count_df = te_video_id_df_rmdup.groupby('user_id').count().reset_index().rename(columns={'video_id': 'video_id_count'})

te_video_author_id_df = pd.merge(te_video_id_count_df,te_author_id_count_df, how='left',on='user_id')
te_video_author_id_df['video_author_ratio'] = te_video_author_id_df['video_id_count']/te_video_author_id_df['author_id_count']
#te_video_author_id_df.head()


# merge register, video, activity

te_x_df = pd.merge(te_x_df, te_launch_num_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_launch_count_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_launch_firstday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_launch_lastday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_launch_interval_df, how='left', on='user_id')

te_x_df = pd.merge(te_x_df, te_video_num_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_video_count_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_video_firstday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_video_lastday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_video_interval_df, how='left', on='user_id')

te_x_df = pd.merge(te_x_df, te_activity_num_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_activity_count_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_activity_firstday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_activity_lastday_interval_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_activity_interval_df, how='left', on='user_id')

te_x_df = pd.merge(te_x_df, te_page_dummies_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_action_type_dummies_df, how='left', on='user_id')
te_x_df = pd.merge(te_x_df, te_video_author_id_df, how='left', on='user_id')

te_x_df[['launch_firstday_interval', 'launch_lastday_interval', 'launch_interval_day',]].fillna(te_time, inplace=True)
te_x_df[['create_firstday_interval', 'create_lastday_interval', 'create_interval_day',]].fillna(te_time, inplace=True)
te_x_df[['activity_firstday_interval', 'activity_lastday_interval', 'activity_interval_day']].fillna(te_time, inplace=True)
te_x_df.fillna(0, inplace=True)

# fill null value with 0 , get_dummie and drop columns
#launch interval day (null value)
te_x_df = pd.concat([te_x_df, pd.get_dummies(te_x_df['register_type'], prefix='register_type')], axis=1)
te_x_df = pd.concat([te_x_df, pd.get_dummies(te_x_df['device_pop_type'], prefix='device_pop_type')], axis=1)
te_x_df = te_x_df.drop(['user_id', 'device_type', 'device_pop_type','register_type'], axis=1)
te_x_df.head()

In [ ]:
# preprocess 
tr_x = np.asarray(tr_x_df)
te_x = np.asarray(te_x_df)

# Normalization
from sklearn import preprocessing
std_scaler_te = preprocessing.StandardScaler().fit(te_x)
std_scaler_tr = preprocessing.StandardScaler().fit(tr_x)
std_te_x_norm = std_scaler_te.transform(te_x)
std_tr_x_norm = std_scaler_tr.transform(tr_x)

#minmax_scaler_te = preprocessing.MinMaxScaler().fit(te_x)
#minmax_scaler_tr = preprocessing.MinMaxScaler().fit(tr_x)
#minmax_te_x_norm = std_scaler_te.transform(te_x)
#minmax_tr_x_norm = std_scaler_tr.transform(tr_x)

In [ ]:
import xgboost as xgb

In [ ]:
xgtrain = xgb.DMatrix(tr_x_norm, tr_y)
xgtest = xgb.DMatrix(te_x_norm)
# specify parameters via map
param = {'max_depth':10, 'n_estimators = 500', 'learning_rate' = 0.05, 'objective':'binary:logistic' }
num_round = 100
bst = xgb.train(param, xgtrain, num_round)
# make prediction
xgb_preds = bst.predict(xgtest)
results = pd.DataFrame(te_register_df['user_id'])
results['pred'] = xgb_preds
actuser = results[results.pred>0.5].user_id.unique()
np.savetxt('xgb.v3.txt', actuser, fmt='%d')